In [99]:
# imports
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
import pandas as pd
from scipy import stats
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from urllib.request import urlopen
import json
import dill

In [100]:
# pre-2019 data
pre_2019=pd.read_csv('skater_stats.csv',encoding = "ISO-8859-1")

In [101]:
# url for scraping 2019 season
url='https://www.hockey-reference.com/leagues/NHL_2019_skaters.html'

In [102]:
# assigning variable to requests getting this url
r=requests.get(url)

In [103]:
# creating soup object
soup = BeautifulSoup(r.content,'html.parser')

In [104]:
#soup.prettify

In [105]:
# gathering information on the players I need data from
player_list=[]
for player in soup.find_all('tbody'):
    player_list.append(player.find_all('tr'))

In [106]:
#player_list

In [107]:
# when I pulled the data it came as a nested list so I need to step inside that list
player_list=player_list[0]

In [108]:
# quick view at what one player looks like
player_list[0]

<tr><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="Abdelkader,Justin" data-append-csv="abdelju01" data-stat="player"><a href="/players/a/abdelju01.html">Justin Abdelkader</a></td><td class="right" data-stat="age">31</td><td class="left" data-stat="team_id"><a href="/teams/DET/2019.html">DET</a></td><td class="center" csk="1:Abdelkader:Justin" data-stat="pos">LW</td><td class="right" data-stat="games_played">71</td><td class="right" data-stat="goals">6</td><td class="right" data-stat="assists">13</td><td class="right" data-stat="points">19</td><td class="right" data-stat="plus_minus">-14</td><td class="right" data-stat="pen_min">38</td><td class="right" data-stat="ps">0.1</td><td class="right" data-stat="goals_ev">4</td><td class="right" data-stat="goals_pp">1</td><td class="right" data-stat="goals_sh">1</td><td class="right iz" data-stat="goals_gw">0</td><td class="right" data-stat="assists_ev">12</td><td class="right" data-stat="assists_pp">1</td>

In [109]:
# total amount of players - sort of - there are repeats that will be discussed further later on
len(player_list)

1121

In [110]:
# creating list of attributes for first player
a=[]
for stat in player_list[0].find_all('td'):
    a.append(stat.text)

In [111]:
# creating list of attributes for every player
b =[]
for i in range(0,1121):
    for stat in player_list[i].find_all('td'):
        b.append(stat.text)

In [112]:
# quick look at beginning of b - what we see is that it captured a little too much and I will need to break this into pieces
b[0:100]

['Justin Abdelkader',
 '31',
 'DET',
 'LW',
 '71',
 '6',
 '13',
 '19',
 '-14',
 '38',
 '0.1',
 '4',
 '1',
 '1',
 '0',
 '12',
 '1',
 '0',
 '95',
 '6.3',
 '1093',
 '15:24',
 '34',
 '185',
 '52',
 '51',
 '50.5',
 'Pontus Aberg',
 '25',
 'TOT',
 'LW',
 '59',
 '12',
 '13',
 '25',
 '-14',
 '20',
 '2.0',
 '9',
 '3',
 '0',
 '1',
 '9',
 '4',
 '0',
 '101',
 '11.9',
 '861',
 '14:36',
 '11',
 '45',
 '2',
 '17',
 '10.5',
 'Pontus Aberg',
 '25',
 'ANA',
 'LW',
 '37',
 '11',
 '8',
 '19',
 '-10',
 '14',
 '1.8',
 '8',
 '3',
 '0',
 '1',
 '7',
 '1',
 '0',
 '74',
 '14.9',
 '578',
 '15:37',
 '7',
 '31',
 '2',
 '9',
 '18.2',
 'Pontus Aberg',
 '25',
 'MIN',
 'LW',
 '22',
 '1',
 '5',
 '6',
 '-4',
 '6',
 '0.2',
 '1',
 '0',
 '0',
 '0',
 '2',
 '3',
 '0',
 '27']

In [113]:
# gauge of how long each player's stats are
b[0:27]

['Justin Abdelkader',
 '31',
 'DET',
 'LW',
 '71',
 '6',
 '13',
 '19',
 '-14',
 '38',
 '0.1',
 '4',
 '1',
 '1',
 '0',
 '12',
 '1',
 '0',
 '95',
 '6.3',
 '1093',
 '15:24',
 '34',
 '185',
 '52',
 '51',
 '50.5']

In [114]:
# list comprehension to split up the list of everything into individual players
chunks = [b[x:x+27] for x in range(0, len(b), 27)]

In [115]:
# what does the first player look like - notice how this is a list within a list at index 0
chunks[0]

['Justin Abdelkader',
 '31',
 'DET',
 'LW',
 '71',
 '6',
 '13',
 '19',
 '-14',
 '38',
 '0.1',
 '4',
 '1',
 '1',
 '0',
 '12',
 '1',
 '0',
 '95',
 '6.3',
 '1093',
 '15:24',
 '34',
 '185',
 '52',
 '51',
 '50.5']

In [116]:
# what does every number correspond to?
for stat in player_list[0].find_all(class_='right'):
    print(stat['data-stat'],stat.text)

ranker 1
age 31
games_played 71
goals 6
assists 13
points 19
plus_minus -14
pen_min 38
ps 0.1
goals_ev 4
goals_pp 1
goals_sh 1
goals_gw 0
assists_ev 12
assists_pp 1
assists_sh 0
shots 95
shot_pct 6.3
time_on_ice 1093
time_on_ice_avg 15:24
blocks 34
hits 185
faceoff_wins 52
faceoff_losses 51


In [117]:
# instantiating data frame to hold data
df=pd.DataFrame(columns=['name','age','team','pos','gp','goals','assists','points','plusmin','plm','ps','goals_ev','goals_pp','goals_sh',
                        'goals_gw','assists_ev','assists_pp','assists_sh','shots','shot_pct','toi','toi_avg',
                        'hits','faceoff_wins','faceoff_losses','faceoff_percent'],
               index=range(0,1076))

In [118]:
# quick look at the empty data frame
df

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_pp,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
# features
len(df.columns)

26

In [120]:
# number of players to populate this list
len(chunks)

1076

In [121]:
# adding name to data frame
for i in range(len(chunks)):
    df.name[i] = chunks[i][0]

In [122]:
# looking for any change in data frame shape
df.shape

(1076, 26)

In [123]:
# double checking that adding the name worked
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_pp,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent
0,Justin Abdelkader,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pontus Aberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pontus Aberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pontus Aberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vitaly Abramov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# this number corresponds to blocks which will be important later
chunks[0][22]

'34'

In [125]:
# adding everything to the data frame
for i in range(len(df)):
    df.name[i] = chunks[i][0]
    df.age[i]=chunks[i][1]
    df.team[i]=chunks[i][2]
    df.pos[i]=chunks[i][3]
    df.gp[i]=chunks[i][4]
    df.goals[i]=chunks[i][5]
    df.assists[i]=chunks[i][6]
    df.points[i]=chunks[i][7]
    df.plusmin[i]=chunks[i][8]
    df.plm[i]=chunks[i][9]
    df.ps[i]=chunks[i][10]
    df.goals_ev[i]=chunks[i][11]
    df.goals_pp[i]=chunks[i][12]
    df.goals_sh[i]=chunks[i][13]
    df.goals_gw[i]=chunks[i][14]
    df.assists_ev[i]=chunks[i][15]
    df.assists_pp[i]=chunks[i][16]
    df.assists_sh[i]=chunks[i][17]
    df.shots[i]=chunks[i][18]
    df.shot_pct[i]=chunks[i][19]
    df.toi[i]=chunks[i][20]
    df.toi_avg[i]=chunks[i][21]
    df.hits[i]=chunks[i][23]
    df.faceoff_wins[i]=chunks[i][24]
    df.faceoff_losses[i]=chunks[i][25]
    df.faceoff_percent[i]=chunks[i][26]

In [126]:
# quick look at data - notice I have repeating name values
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_pp,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,1,0,95,6.3,1093,15:24,185,52,51,50.5
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,4,0,101,11.9,861,14:36,45,2,17,10.5
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,1,0,74,14.9,578,15:37,31,2,9,18.2
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,3,0,27,3.7,283,12:52,14,0,8,0.0
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0,0,0,,14,13:52,0,0,0,


In [127]:
# populating list of blocks by player
block_list = []
for i in range(len(chunks)):
    block_list.append(int(chunks[i][22]))

In [128]:
# quick look at list of blocks by player
block_list[0:5]

[34, 11, 7, 4, 1]

In [129]:
# length of list of blocks by player
len(block_list)

1076

In [130]:
# adding block statistic to data frame - for some reason this stat was very hard to add so I need to do it in a roundabout way
df['block']=0
for i in range(len(df)):
    df.block[i]=block_list[i]

In [131]:
# overview of data
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent,block
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,0,95,6.3,1093,15:24,185,52,51,50.5,34
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,0,101,11.9,861,14:36,45,2,17,10.5,11
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,0,74,14.9,578,15:37,31,2,9,18.2,7
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,0,27,3.7,283,12:52,14,0,8,0.0,4
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0,0,,14,13:52,0,0,0,,1


In [132]:
# tail of data
df.tail()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent,block
1071,Jason Zucker,27,MIN,LW,81,21,21,42,-9,28,...,0,214,9.8,1383,17:05,87,2,11,15.4,38
1072,Valentin Zykov,23,TOT,LW,28,2,3,5,-5,2,...,0,29,6.9,279,9:57,26,2,7,22.2,6
1073,Valentin Zykov,23,CAR,LW,13,0,3,3,-1,0,...,0,13,0.0,127,9:45,6,2,6,25.0,2
1074,Valentin Zykov,23,VEG,LW,10,2,0,2,-3,0,...,0,16,12.5,116,11:37,18,0,1,0.0,3
1075,Valentin Zykov,23,EDM,LW,5,0,0,0,-1,2,...,0,0,,36,7:08,2,0,0,,1


In [133]:
# data info - notice I will need to change a lot of numbers to be integers or floats
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 27 columns):
name               1076 non-null object
age                1076 non-null object
team               1076 non-null object
pos                1076 non-null object
gp                 1076 non-null object
goals              1076 non-null object
assists            1076 non-null object
points             1076 non-null object
plusmin            1076 non-null object
plm                1076 non-null object
ps                 1076 non-null object
goals_ev           1076 non-null object
goals_pp           1076 non-null object
goals_sh           1076 non-null object
goals_gw           1076 non-null object
assists_ev         1076 non-null object
assists_pp         1076 non-null object
assists_sh         1076 non-null object
shots              1076 non-null object
shot_pct           1076 non-null object
toi                1076 non-null object
toi_avg            1076 non-null object
hits         

In [134]:
# mapping age as an integer
df.age=df.age.astype(int)

In [135]:
# mapping games played as an integer
df.gp=df.gp.astype(int)

In [136]:
# mapping other features as integer or float - some gave me trouble and have been commented out
df.goals=df.goals.astype(int)
df.goals_ev=df.goals_ev.astype(int)
df.goals_gw=df.goals_gw.astype(int)
df.goals_pp=df.goals_pp.astype(int)
df.goals_sh=df.goals_sh.astype(int)
df.assists=df.assists.astype(int)
df.assists_ev=df.assists_ev.astype(int)
df.assists_pp=df.assists_pp.astype(int)
df.assists_sh=df.assists_sh.astype(int)
df.points=df.points.astype(int)
df.plusmin=df.plusmin.astype(int)
df.plm=df.plm.astype(int)
df.ps=df.ps.astype(float)
df.shots=df.shots.astype(float)
#df.shot_pct=df.shot_pct.astype(float)
#df.toi=df.toi.astype(float)
#df.toi_avg=df.toi_avg.astype(float)
df.hits=df.hits.astype(int)
df.faceoff_wins=df.faceoff_wins.astype(int)
df.faceoff_losses=df.faceoff_losses.astype(int)
#df.faceoff_percent=df.faceoff_percent.astype(float)

In [137]:
# review of data
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,assists_sh,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,faceoff_percent,block
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,0,95.0,6.3,1093,15:24,185,52,51,50.5,34
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,0,101.0,11.9,861,14:36,45,2,17,10.5,11
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,0,74.0,14.9,578,15:37,31,2,9,18.2,7
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,0,27.0,3.7,283,12:52,14,0,8,0.0,4
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0,0.0,,14,13:52,0,0,0,,1


In [138]:
# remove faceoff percent
df.drop('faceoff_percent',axis=1,inplace=True)

In [139]:
# add faceoff percent back as a float
df['faceoff_pct']=round(df.faceoff_wins/(df.faceoff_losses+df.faceoff_wins),3)

In [140]:
# adding season column so I can later add more data
df['season']=2019

In [141]:
# review of data
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,shots,shot_pct,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,95.0,6.3,1093,15:24,185,52,51,34,0.505,2019
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,101.0,11.9,861,14:36,45,2,17,11,0.105,2019
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,74.0,14.9,578,15:37,31,2,9,7,0.182,2019
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,27.0,3.7,283,12:52,14,0,8,4,0.000,2019
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0.0,,14,13:52,0,0,0,1,NaN,2019


In [142]:
#df.shot_pct=df.shot_pct.astype(float)
#df.toi=df.toi.astype(float)
#df.toi_avg=df.toi_avg.astype(float)

In [143]:
# removing shot percent
df.drop('shot_pct',axis=1,inplace=True)

In [144]:
# adding shot percent back as a float
df['shot_pct']=df.goals/df.shots

In [145]:
# time on ice gave me trouble so I am going to have to break this down
df.toi_avg

0       15:24
1       14:36
2       15:37
3       12:52
4       13:52
        ...  
1071    17:05
1072     9:57
1073     9:45
1074    11:37
1075     7:08
Name: toi_avg, Length: 1076, dtype: object

In [146]:
# first instance of toi
df.toi_avg[0]

'15:24'

In [147]:
# length of toi "type 1"
len(df.toi_avg[0])

5

In [148]:
# minutes component
df.toi_avg[0].split()[0][0:2]

'15'

In [149]:
# seconds component
df.toi_avg[0].split()[0][3:5]

'24'

In [150]:
# toi "type 2"

df.toi_avg[1073]

'9:45'

In [151]:
# length of toi "type 2"
len(df.toi_avg[1073])

4

In [152]:
# minutes
df.toi_avg[1073].split()[0][0:1]

'9'

In [153]:
# seconds
df.toi_avg[1073].split()[0][2:4]

'45'

In [154]:
# converting minutes and seconds to total seconds
for i in range(len(df)):
    if len(df.toi_avg[i]) == 5:
        df.toi_avg[i] = int(df.toi_avg[i].split()[0][0:2])*60 +int(df.toi_avg[i].split()[0][3:5])
    elif len(df.toi_avg[i]) == 4:
        df.toi_avg[i] = int(df.toi_avg[i].split()[0][0:1])*60 +int(df.toi_avg[i].split()[0][2:4])

In [155]:
# review data having made some modifications
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,shots,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,95.0,1093,924,185,52,51,34,0.505,2019,0.063158
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,101.0,861,876,45,2,17,11,0.105,2019,0.118812
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,74.0,578,937,31,2,9,7,0.182,2019,0.148649
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,27.0,283,772,14,0,8,4,0.000,2019,0.037037
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0.0,14,832,0,0,0,1,NaN,2019,NaN


In [156]:
# data types for data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 28 columns):
name              1076 non-null object
age               1076 non-null int64
team              1076 non-null object
pos               1076 non-null object
gp                1076 non-null int64
goals             1076 non-null int64
assists           1076 non-null int64
points            1076 non-null int64
plusmin           1076 non-null int64
plm               1076 non-null int64
ps                1076 non-null float64
goals_ev          1076 non-null int64
goals_pp          1076 non-null int64
goals_sh          1076 non-null int64
goals_gw          1076 non-null int64
assists_ev        1076 non-null int64
assists_pp        1076 non-null int64
assists_sh        1076 non-null int64
shots             1076 non-null float64
toi               1076 non-null object
toi_avg           1076 non-null object
hits              1076 non-null int64
faceoff_wins      1076 non-null int64
faceoff_lo

In [157]:
# toi as a float
df.toi_avg=df.toi_avg.astype(int)

In [158]:
# review data
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,shots,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,95.0,1093,924,185,52,51,34,0.505,2019,0.063158
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,101.0,861,876,45,2,17,11,0.105,2019,0.118812
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,74.0,578,937,31,2,9,7,0.182,2019,0.148649
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,27.0,283,772,14,0,8,4,0.000,2019,0.037037
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0.0,14,832,0,0,0,1,NaN,2019,NaN


In [159]:
# data types for data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 28 columns):
name              1076 non-null object
age               1076 non-null int64
team              1076 non-null object
pos               1076 non-null object
gp                1076 non-null int64
goals             1076 non-null int64
assists           1076 non-null int64
points            1076 non-null int64
plusmin           1076 non-null int64
plm               1076 non-null int64
ps                1076 non-null float64
goals_ev          1076 non-null int64
goals_pp          1076 non-null int64
goals_sh          1076 non-null int64
goals_gw          1076 non-null int64
assists_ev        1076 non-null int64
assists_pp        1076 non-null int64
assists_sh        1076 non-null int64
shots             1076 non-null float64
toi               1076 non-null object
toi_avg           1076 non-null int64
hits              1076 non-null int64
faceoff_wins      1076 non-null int64
faceoff_los

In [160]:
# how many unique players are there?
df.groupby('name').sum().shape

(906, 24)

In [161]:
# review data
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,shots,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,95.0,1093,924,185,52,51,34,0.505,2019,0.063158
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,101.0,861,876,45,2,17,11,0.105,2019,0.118812
2,Pontus Aberg,25,ANA,LW,37,11,8,19,-10,14,...,74.0,578,937,31,2,9,7,0.182,2019,0.148649
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,27.0,283,772,14,0,8,4,0.000,2019,0.037037
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,0.0,14,832,0,0,0,1,NaN,2019,NaN


In [162]:
# review data I will later merge
pre_2019.head()

,Unnamed: 0,Season,Player,Age,Tm,Pos,GP,G,GPG,A,...,SHA,S,S%,TOI,ATOI,BLK,HIT,FOwin,FOloss,FO%
0,2018Justin Abdelkader,2018,Justin Abdelkader,30.0,DET,LW,75,13,0.1733,22,...,-,110,12,"1,241",16:33,40.0,174.0,47.0,50.0,48.5
1,2018Pontus Aberg,2018,Pontus Aberg,24.0,TOT,LW,53,4,0.0755,12,...,-,70,6,645,12:10,8.0,24.0,4.0,8.0,33.3
2,2018Noel Acciari,2018,Noel Acciari,26.0,BOS,C,60,10,0.1667,1,...,-,66,15,775,12:55,41.0,152.0,42.0,51.0,45.2
3,2018Kenny Agostino,2018,Kenny Agostino,25.0,BOS,LW,5,-,0.0000,1,...,-,11,-,60,12:03,1.0,4.0,0.0,1.0,0.0
4,2018Sebastian Aho,2018,Sebastian Aho,20.0,CAR,RW,78,29,0.3718,36,...,-,200,15,"1,398",17:55,17.0,65.0,78.0,94.0,45.3


In [163]:
# checking names of columns in 2019 season
df.columns

Index(['name', 'age', 'team', 'pos', 'gp', 'goals', 'assists', 'points',
       'plusmin', 'plm', 'ps', 'goals_ev', 'goals_pp', 'goals_sh', 'goals_gw',
       'assists_ev', 'assists_pp', 'assists_sh', 'shots', 'toi', 'toi_avg',
       'hits', 'faceoff_wins', 'faceoff_losses', 'block', 'faceoff_pct',
       'season', 'shot_pct'],
      dtype='object')

In [164]:
# checking names of columns in 2018 season
pre_2019.columns

Index(['Unnamed: 0', 'Season', 'Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'GPG',
       'A', 'PTS', '+/-', 'PIM', 'EVG', 'PPG', 'SHG', 'GWG', 'EVA', 'PPA',
       'SHA', 'S', 'S%', 'TOI', 'ATOI', 'BLK', 'HIT', 'FOwin', 'FOloss',
       'FO%'],
      dtype='object')

In [165]:
# adding stat missing from 2019 season
df['gpg']=df.goals/df.gp

In [166]:
# removing pointless column
pre_2019.drop('Unnamed: 0',axis=1,inplace=True)

In [167]:
# do the data frames align?
len(df.columns)==len(pre_2019.columns)

False

In [168]:
# checking for difference
pre_2019.columns

Index(['Season', 'Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'GPG', 'A', 'PTS',
       '+/-', 'PIM', 'EVG', 'PPG', 'SHG', 'GWG', 'EVA', 'PPA', 'SHA', 'S',
       'S%', 'TOI', 'ATOI', 'BLK', 'HIT', 'FOwin', 'FOloss', 'FO%'],
      dtype='object')

In [169]:
# checking for difference
df.columns

Index(['name', 'age', 'team', 'pos', 'gp', 'goals', 'assists', 'points',
       'plusmin', 'plm', 'ps', 'goals_ev', 'goals_pp', 'goals_sh', 'goals_gw',
       'assists_ev', 'assists_pp', 'assists_sh', 'shots', 'toi', 'toi_avg',
       'hits', 'faceoff_wins', 'faceoff_losses', 'block', 'faceoff_pct',
       'season', 'shot_pct', 'gpg'],
      dtype='object')

In [170]:
# removing the extra column
df.drop('ps',axis=1,inplace=True)

In [171]:
# reorganizing the order of pre-2019 data columns
pre_2019_2 = pre_2019[['Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'A', 'PTS','+/-', 'PIM', 'EVG', 'PPG', 'SHG', 
                       'GWG', 'EVA', 'PPA', 'SHA', 'S', 'TOI', 'ATOI', 'HIT', 'FOwin', 'FOloss', 
                       'BLK', 'FO%', 'Season', 'S%', 'GPG']]

In [172]:
# assigning uniform name to columns of separate data frames
pre_2019_2.columns=df.columns

In [173]:
# data of pre-2019 season with new feature names
pre_2019_2.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
0,Justin Abdelkader,30.0,DET,LW,75,13,22,35,-11,78,...,"1,241",16:33,174.0,47.0,50.0,40.0,48.5,2018,12,0.1733
1,Pontus Aberg,24.0,TOT,LW,53,4,12,16,9,10,...,645,12:10,24.0,4.0,8.0,8.0,33.3,2018,6,0.0755
2,Noel Acciari,26.0,BOS,C,60,10,1,11,-6,9,...,775,12:55,152.0,42.0,51.0,41.0,45.2,2018,15,0.1667
3,Kenny Agostino,25.0,BOS,LW,5,-,1,1,-1,4,...,60,12:03,4.0,0.0,1.0,1.0,0.0,2018,-,0.0000
4,Sebastian Aho,20.0,CAR,RW,78,29,36,65,4,24,...,"1,398",17:55,65.0,78.0,94.0,17.0,45.3,2018,15,0.3718


In [174]:
# figuring out which players changed teams
df[df.name.duplicated()==True].name.unique()

array(['Pontus Aberg', 'Kenny Agostino', 'Darren Archibald',
       'Nathan Beaulieu', 'Anthony Bitetto', 'Nick Bjugstad',
       'Joseph Blandisi', 'Madison Bowey', 'Brian Boyle',
       'Derick Brassard', 'Drake Caggiula', 'Paul Carey',
       'Connor Carrick', 'Colby Cave', 'Andrew Cogliano', 'Charlie Coyle',
       'Jean-Sebastien Dea', 'Michael Del Zotto', 'Phillip Di Giuseppe',
       'Ryan Donato', 'Matt Duchene', 'Anthony Duclair', 'Ryan Dzingel',
       'Oscar Fantenberg', 'Kevin Fiala', 'Christian Folin', 'Sam Gagner',
       'Brian Gibbons', 'Mikael Granlund', 'Derek Grant',
       'Erik Gudbranson', 'Brendan Guhle', 'Carl Hagelin',
       'Micheal Haley', 'Ryan Hartman', 'Kevin Hayes', 'Matt Hendricks',
       'Brad Hunt', 'Nick Jensen', 'Marcus Johansson', 'Slater Koekkoek',
       'Brendan Leipsic', 'Josh Leivo', 'Brendan Lemieux',
       'Oscar Lindberg', 'Par Lindholm', 'Ben Lovejoy', 'Brandon Manning',
       'Jared McCann', 'Cody McLeod', 'Adam McQuaid', 'Brandon Mont

In [175]:
# checking how traded players are represented
df[df.name=='Ryan Strome']

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
963,Ryan Strome,25,TOT,C,81,19,16,35,-3,64,...,1261,934,76,365,402,36,0.476,2019,0.172727,0.234568
964,Ryan Strome,25,NYR,C,63,18,15,33,-2,50,...,1002,954,53,266,298,31,0.472,2019,0.225000,0.285714
965,Ryan Strome,25,EDM,C,18,1,1,2,-1,14,...,259,865,23,99,104,5,0.488,2019,0.033333,0.055556


In [176]:
# creating data frame with only the final team that player was on - I intend to declare the final team played for as that
# player's team for the season. This way I get a full picture of things like goals and points
final_team_df = df[df.name.duplicated()==True].drop_duplicates(subset='name',keep='last')

In [177]:
# where did "repeat" players end their season
final_team_df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
3,Pontus Aberg,25,MIN,LW,22,1,5,6,-4,6,...,283,772,14,0,8,4,0.00,2019,0.037037,0.045455
8,Kenny Agostino,26,NJD,LW,27,4,9,13,-2,8,...,411,913,63,7,13,10,0.35,2019,0.097561,0.148148
24,Darren Archibald,28,OTT,RW,3,0,0,0,-2,0,...,20,395,2,0,0,1,NaN,2019,0.000000,0.000000
52,Nathan Beaulieu,26,WPG,D,18,0,5,5,5,7,...,303,1011,27,0,0,12,NaN,2019,0.000000,0.000000
68,Anthony Bitetto,28,MIN,D,18,0,0,0,-8,4,...,237,791,40,0,0,13,NaN,2019,0.000000,0.000000


In [178]:
# reset index of this data frame
final_team_df.reset_index(inplace=True)

In [179]:
# drop pointless column
final_team_df.drop('index',axis=1,inplace=True)

In [180]:
# create dictionary to assign values to team name 
final_team_dict = {}
for i in range(len(final_team_df)):
    final_team_dict[final_team_df.name[i]]=final_team_df.team[i]

In [181]:
# quick look at dictionary beginning
list(final_team_dict.items())[0:15]

[('Pontus Aberg', 'MIN'),
 ('Kenny Agostino', 'NJD'),
 ('Darren Archibald', 'OTT'),
 ('Nathan Beaulieu', 'WPG'),
 ('Anthony Bitetto', 'MIN'),
 ('Nick Bjugstad', 'FLA'),
 ('Joseph Blandisi', 'ANA'),
 ('Madison Bowey', 'DET'),
 ('Brian Boyle', 'NSH'),
 ('Derick Brassard', 'FLA'),
 ('Drake Caggiula', 'CHI'),
 ('Paul Carey', 'BOS'),
 ('Connor Carrick', 'DAL'),
 ('Colby Cave', 'BOS'),
 ('Andrew Cogliano', 'DAL')]

In [182]:
# quick look at dictionary beginning
list(final_team_dict.keys())[0:5]

['Pontus Aberg',
 'Kenny Agostino',
 'Darren Archibald',
 'Nathan Beaulieu',
 'Anthony Bitetto']

In [183]:
# removing repeats of the same player and keeping only total season stats (TOT)
df=df.drop_duplicates(subset='name',keep='first')

In [184]:
# new look of data
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,1093,924,185,52,51,34,0.505,2019,0.063158,0.084507
1,Pontus Aberg,25,TOT,LW,59,12,13,25,-14,20,...,861,876,45,2,17,11,0.105,2019,0.118812,0.203390
4,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,14,832,0,0,0,1,NaN,2019,NaN,0.000000
5,Noel Acciari,27,BOS,C,72,6,8,14,-3,47,...,935,779,221,200,203,36,0.496,2019,0.060606,0.083333
6,Kenny Agostino,26,TOT,LW,63,6,18,24,-3,34,...,814,775,143,15,27,14,0.357,2019,0.075949,0.095238


In [185]:
# insuring that name is a string and not object
df.name = df.name.astype(str)

In [186]:
# cheking that my dictionary will map well
'Pontus Aberg' in list(final_team_dict.keys())

True

In [187]:
# reset index to deal with players removed
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [188]:
# replace "TOT" with the final team that player played for
for i in range(len(df)):
    if df.name.iloc[i] in list(final_team_dict.keys()):
        df.team.iloc[i] = final_team_dict[df.name.iloc[i]]

In [189]:
# head of data
df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
0,Justin Abdelkader,31,DET,LW,71,6,13,19,-14,38,...,1093,924,185,52,51,34,0.505,2019,0.063158,0.084507
1,Pontus Aberg,25,MIN,LW,59,12,13,25,-14,20,...,861,876,45,2,17,11,0.105,2019,0.118812,0.203390
2,Vitaly Abramov,20,OTT,RW,1,0,0,0,-3,0,...,14,832,0,0,0,1,NaN,2019,NaN,0.000000
3,Noel Acciari,27,BOS,C,72,6,8,14,-3,47,...,935,779,221,200,203,36,0.496,2019,0.060606,0.083333
4,Kenny Agostino,26,NJD,LW,63,6,18,24,-3,34,...,814,775,143,15,27,14,0.357,2019,0.075949,0.095238


In [190]:
# tail of data
df.tail()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
901,Jakub Zboril,21,BOS,D,2,0,0,0,0,0,...,22,656,3,0,0,0,NaN,2019,0.000000,0.000000
902,Mika Zibanejad,25,NYR,C,82,30,44,74,-12,47,...,1686,1234,134,830,842,66,0.496,2019,0.127119,0.365854
903,Mats Zuccarello,31,DAL,LW,48,12,28,40,-8,24,...,953,1191,57,10,20,43,0.333,2019,0.115385,0.250000
904,Jason Zucker,27,MIN,LW,81,21,21,42,-9,28,...,1383,1025,87,2,11,38,0.154,2019,0.098131,0.259259
905,Valentin Zykov,23,EDM,LW,28,2,3,5,-5,2,...,279,597,26,2,7,6,0.222,2019,0.068966,0.071429


In [191]:
# data frame to combine with
pre_2019_2.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
0,Justin Abdelkader,30.0,DET,LW,75,13,22,35,-11,78,...,"1,241",16:33,174.0,47.0,50.0,40.0,48.5,2018,12,0.1733
1,Pontus Aberg,24.0,TOT,LW,53,4,12,16,9,10,...,645,12:10,24.0,4.0,8.0,8.0,33.3,2018,6,0.0755
2,Noel Acciari,26.0,BOS,C,60,10,1,11,-6,9,...,775,12:55,152.0,42.0,51.0,41.0,45.2,2018,15,0.1667
3,Kenny Agostino,25.0,BOS,LW,5,-,1,1,-1,4,...,60,12:03,4.0,0.0,1.0,1.0,0.0,2018,-,0.0000
4,Sebastian Aho,20.0,CAR,RW,78,29,36,65,4,24,...,"1,398",17:55,65.0,78.0,94.0,17.0,45.3,2018,15,0.3718


In [192]:
# data frame to combine with
pre_2019_2.tail()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
37820,Gordie Drillon*,26.0,TOR,RW,43,21,19,40,NaN,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.4884
37821,Bill Cowley*,27.0,BOS,C,48,13,27,40,NaN,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.2708
37822,Woody Dumart*,23.0,BOS,LW,48,22,21,43,NaN,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.4583
37823,Bobby Bauer*,24.0,BOS,RW,48,17,26,43,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.3542
37824,Milt Schmidt*,21.0,BOS,C,48,22,30,52,NaN,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.4583


In [193]:
total_df=pd.concat([df,pre_2019_2])

In [194]:
total_df.head()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
0,Justin Abdelkader,31.0,DET,LW,71,6,13,19,-14,38,...,1093,924,185.0,52.0,51.0,34.0,0.505,2019,0.0631579,0.084507
1,Pontus Aberg,25.0,MIN,LW,59,12,13,25,-14,20,...,861,876,45.0,2.0,17.0,11.0,0.105,2019,0.118812,0.203390
2,Vitaly Abramov,20.0,OTT,RW,1,0,0,0,-3,0,...,14,832,0.0,0.0,0.0,1.0,NaN,2019,NaN,0.000000
3,Noel Acciari,27.0,BOS,C,72,6,8,14,-3,47,...,935,779,221.0,200.0,203.0,36.0,0.496,2019,0.0606061,0.083333
4,Kenny Agostino,26.0,NJD,LW,63,6,18,24,-3,34,...,814,775,143.0,15.0,27.0,14.0,0.357,2019,0.0759494,0.095238


In [195]:
total_df.tail()

,name,age,team,pos,gp,goals,assists,points,plusmin,plm,...,toi,toi_avg,hits,faceoff_wins,faceoff_losses,block,faceoff_pct,season,shot_pct,gpg
37820,Gordie Drillon*,26.0,TOR,RW,43,21,19,40,NaN,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.4884
37821,Bill Cowley*,27.0,BOS,C,48,13,27,40,NaN,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.2708
37822,Woody Dumart*,23.0,BOS,LW,48,22,21,43,NaN,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.4583
37823,Bobby Bauer*,24.0,BOS,RW,48,17,26,43,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.3542
37824,Milt Schmidt*,21.0,BOS,C,48,22,30,52,NaN,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1940,NaN,0.4583


In [196]:
total_df.columns

Index(['name', 'age', 'team', 'pos', 'gp', 'goals', 'assists', 'points',
       'plusmin', 'plm', 'goals_ev', 'goals_pp', 'goals_sh', 'goals_gw',
       'assists_ev', 'assists_pp', 'assists_sh', 'shots', 'toi', 'toi_avg',
       'hits', 'faceoff_wins', 'faceoff_losses', 'block', 'faceoff_pct',
       'season', 'shot_pct', 'gpg'],
      dtype='object')

In [198]:
total_df=total_df[total_df.season>1979]

In [200]:
total_df.to_csv('skater_data.csv')

# I need to add two or three goalies for each year!

# So that is in a separate notebook, but just for fun I have added a scraper function for skaters

In [220]:
def create_skater_dataframe(year):
    url=f'https://www.hockey-reference.com/leagues/NHL_{year}_skaters.html'
    r=requests.get(url)
    soup = BeautifulSoup(r.content,'html.parser')
    player_list=[]
    for player in soup.find_all('tbody'):
        player_list.append(player.find_all('tr'))
    player_list=player_list[0]
    b =[]
    for i in range(0,len(player_list)):
        for stat in player_list[i].find_all('td'):
            b.append(stat.text)
    chunks = [b[x:x+27] for x in range(0, len(b), 27)]
    df=pd.DataFrame(columns=['name','age','team','pos','gp','goals','assists','points','plusmin','plm','ps','goals_ev','goals_pp','goals_sh',
                        'goals_gw','assists_ev','assists_pp','assists_sh','shots','shot_pct','toi','toi_avg',
                        'hits','faceoff_wins','faceoff_losses','faceoff_percent'],
               index=range(0,len(chunks)))
    for i in range(len(df)):
        df.name[i] = chunks[i][0]
        df.age[i]=chunks[i][1]
        df.team[i]=chunks[i][2]
        df.pos[i]=chunks[i][3]
        df.gp[i]=chunks[i][4]
        df.goals[i]=chunks[i][5]
        df.assists[i]=chunks[i][6]
        df.points[i]=chunks[i][7]
        df.plusmin[i]=chunks[i][8]
        df.plm[i]=chunks[i][9]
        df.ps[i]=chunks[i][10]
        df.goals_ev[i]=chunks[i][11]
        df.goals_pp[i]=chunks[i][12]
        df.goals_sh[i]=chunks[i][13]
        df.goals_gw[i]=chunks[i][14]
        df.assists_ev[i]=chunks[i][15]
        df.assists_pp[i]=chunks[i][16]
        df.assists_sh[i]=chunks[i][17]
        df.shots[i]=chunks[i][18]
        df.shot_pct[i]=chunks[i][19]
        df.toi[i]=chunks[i][20]
        df.toi_avg[i]=chunks[i][21]
        df.hits[i]=chunks[i][23]
        df.faceoff_wins[i]=chunks[i][24]
        df.faceoff_losses[i]=chunks[i][25]
        df.faceoff_percent[i]=chunks[i][26]
    df.drop('faceoff_percent',axis=1,inplace=True)
    for f in ['faceoff_wins','faceoff_losses','goals','shots','gp','points']:
        df[f].replace("",0,inplace=True)
        df[f]=df[f].astype(int)
    df['faceoff_pct']=round(df.faceoff_wins/(df.faceoff_losses+df.faceoff_wins),3)
    df['season']=int(year)
    df.drop('shot_pct',axis=1,inplace=True)
    df['shot_pct']=df.goals/df.shots
    for i in range(len(df)):
        if len(df.toi_avg[i]) == 5:
            df.toi_avg[i] = int(df.toi_avg[i].split()[0][0:2])*60 +int(df.toi_avg[i].split()[0][3:5])
        elif len(df.toi_avg[i]) == 4:
            df.toi_avg[i] = int(df.toi_avg[i].split()[0][0:1])*60 +int(df.toi_avg[i].split()[0][2:4])
    df.toi_avg=df.toi_avg.astype(int)
    df['gpg']=df.goals/df.gp
    df['ppg']=df.points/df.gp
    df.drop('ps',axis=1,inplace=True)
    final_team_df = df[df.name.duplicated()==True].drop_duplicates(subset='name',keep='last')
    final_team_df.reset_index(inplace=True)
    final_team_dict = {}
    for i in range(len(final_team_df)):
        final_team_dict[final_team_df.name[i]]=final_team_df.team[i]
    df=df.drop_duplicates(subset='name',keep='first')
    df.name = df.name.astype(str)
    df.reset_index(inplace=True)
    df.drop('index',axis=1,inplace=True)
    for i in range(len(df)):
        if df.name.iloc[i] in list(final_team_dict.keys()):
            df.team.iloc[i] = final_team_dict[df.name.iloc[i]]
    
    return df
            

In [ ]:
create_skater_dataframe('2020')